Problem Statement

Experience-based businesses (concerts, stand-up shows, workshops) struggle to decide what type of event to launch, in which city, in which month, and at what ticket price. Decisions are often made using intuition, which can lead to poor turnout, wrong pricing, and wasted marketing budget.
This project builds an AI-based Experience Market Planner that analyzes historical demand signals (Google Trends + seasonality) and uses city spending power to recommend whether to add a show, keep pricing, increase price, decrease price, or pause/remove the show.

Approach / Architecture (High Level)

Input Data Sources

Google Trends CSVs (multiTimeline 1–35)

Gives monthly interest levels for experience-related keywords (concerts, stand-up, workshops).

Seasonality Dataset

Gives a month-wise weight/score representing seasonal demand patterns.

City Payment Power Multiplier (heuristic)

Adjusts demand based on city spending capacity (Tier 1 vs Tier 2 vs Tier 3).

Pipeline

Data Ingestion

Load all multiTimeline CSVs and seasonality CSV from GitHub.

Preprocessing

Clean trends format, parse dates, extract month/year.

Convert trends data from wide format → long format (Keyword, Trend_Score).

Feature Engineering

Convert seasonality weights into a 0–100 score.

Create a combined Demand_Index using:

Trend_Score (demand signal)

Season_Score (seasonal boost)

Category Mapping

Map 35 keywords into only 3 experience categories:

Concert

Standup

Workshop

City Adjustment

Multiply demand using City Multiplier to simulate real purchasing power differences.

Decision Engine

Uses demand + price thresholds to recommend:

INCREASE PRICE

DECREASE PRICE

ADD SHOWS

KEEP

REMOVE / PAUSE

TEST SMALL

Dashboard (Streamlit)

User selects: Show Type → City → Month → Price

Dashboard outputs: demand snapshot + recommendation + downloadable plan.

Core Features

3-Type Experience Forecasting

Only supports the main business categories:
Concert / Standup / Workshop

Keeps the planner simple and focused.

Month-wise Demand Prediction

Shows average demand for the selected month using past trend patterns + seasonality.

City-Based Pricing Adjustment

Uses a city spending power multiplier so that the same show can have different pricing decisions in different cities.

Pricing Recommendation System

Based on demand + price combination, gives actionable decision:

Increase price / decrease price / add more shows / remove.

Planner Table + Export

Allows adding recommendations into a plan table.

Download planner as CSV for business execution.

Edge Case Robustness

If city is not in mapping -> defaults to multiplier = 1.00

If demand is very low -> recommends test small or pause instead of forcing an event.


In [49]:


import pandas as pd, requests, urllib.parse
from io import StringIO

base = "https://raw.githubusercontent.com/YuvrajDeol/IITRPR_ExperienceMarketAnalaysis/main/Google%20trends%20%202/"

# --- Load Seasonality ---
seasonality = pd.read_csv(base + "Seasonality.csv")
seasonality.columns = [c.strip() for c in seasonality.columns]
print("Seasonality columns:", seasonality.columns.tolist())

# Auto-detect month column
if "Month" in seasonality.columns:
    month_col = "Month"
elif "Month_Num" in seasonality.columns:
    month_col = "Month_Num"
else:
    raise KeyError("No Month column found in Seasonality.csv (expected Month or Month_Num).")

seasonality["Month"] = pd.to_numeric(seasonality[month_col], errors="coerce")
seasonality = seasonality.dropna(subset=["Month"])
seasonality["Month"] = seasonality["Month"].astype(int)

# Ensure Season_Weight exists
if "Season_Weight" not in seasonality.columns:
    raise KeyError("Season_Weight column not found in Seasonality.csv")

# --- Load & clean Google Trends ---
def load_trends(url):
    text = requests.get(url).text
    lines = text.splitlines()
    header_idx = next((i for i, l in enumerate(lines) if l.lower().strip().startswith(("week", "month", "day", "date"))), None)
    cleaned = "\n".join(lines[header_idx:]) if header_idx is not None else text

    df = pd.read_csv(StringIO(cleaned))
    df.columns = [c.strip() for c in df.columns]
    df.rename(columns={df.columns[0]: "Date"}, inplace=True)

    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month

    value_cols = [c for c in df.columns if c not in ["Date", "Year", "Month"]]
    long = df.melt(
        id_vars=["Date", "Year", "Month"],
        value_vars=value_cols,
        var_name="Keyword",
        value_name="Trend_Score"
    )
    long["Trend_Score"] = pd.to_numeric(long["Trend_Score"], errors="coerce")
    long = long.dropna(subset=["Trend_Score"])
    return long

# --- Load multiTimeline (1) to (35) ---
all_trends = []
failed = []

for i in range(1, 36):
    fname = f"multiTimeline ({i}).csv"
    url = base + urllib.parse.quote(fname)
    r = requests.get(url)
    if r.status_code == 200:
        t = load_trends(url)
        t["Source_File"] = fname
        all_trends.append(t)
    else:
        failed.append(fname)

print("Loaded trend files:", len(all_trends))
if failed:
    print("Failed files:", failed)

trends_master = pd.concat(all_trends, ignore_index=True)

# --- Merge with seasonality ---
merged = trends_master.merge(seasonality[["Month", "Season_Weight"]], on="Month", how="left")

# --- NEW MATH: scale Season_Weight to 0–100 before combining ---
season_min = merged["Season_Weight"].min()
season_max = merged["Season_Weight"].max()

if season_max == season_min:
    merged["Season_Score"] = 50.0
else:
    merged["Season_Score"] = ((merged["Season_Weight"] - season_min) / (season_max - season_min)) * 100

# Demand Index (both on same scale now)
merged["Demand_Index"] = 0.7 * merged["Trend_Score"] + 0.3 * merged["Season_Score"]

# Save final dataset
merged.to_csv("/content/final_trends_seasonality.csv", index=False)
print("Saved -> /content/final_trends_seasonality.csv")

# Quick sanity check
print("\nDemand_Index summary:")
print(merged["Demand_Index"].describe())

merged.head()


Seasonality columns: ['Month_Num', 'Season_Weight', 'Reason']
Loaded trend files: 35
Saved -> /content/final_trends_seasonality.csv

Demand_Index summary:
count    1855.000000
mean       39.259931
std        22.328073
min         0.000000
25%        24.545455
50%        33.718182
75%        53.100000
max       100.000000
Name: Demand_Index, dtype: float64


,Date,Year,Month,Keyword,Trend_Score,Source_File,Season_Weight,Season_Score,Demand_Index
0,2024-12-29,2024,12,Music Festival India: (India),15,multiTimeline (1).csv,1.5,100.000000,40.500000
1,2025-01-05,2025,1,Music Festival India: (India),20,multiTimeline (1).csv,1.0,54.545455,30.363636
2,2025-01-12,2025,1,Music Festival India: (India),0,multiTimeline (1).csv,1.0,54.545455,16.363636
3,2025-01-19,2025,1,Music Festival India: (India),15,multiTimeline (1).csv,1.0,54.545455,26.863636
4,2025-01-26,2025,1,Music Festival India: (India),0,multiTimeline (1).csv,1.0,54.545455,16.363636


In [50]:
%%writefile app.py
import streamlit as st
import pandas as pd

st.set_page_config(page_title="Show Forecast Dashboard", layout="wide")

st.title("🎯 Show Forecast Dashboard")
st.caption("Show Type → City → Month → Price → Recommendation (Add / Remove / Increase / Decrease)")

@st.cache_data
def load_data():
    return pd.read_csv("final_trends_seasonality.csv")

df = load_data()

# -----------------------------
# Show Type -> Keyword mapping
# -----------------------------
category_map = {
    "Concert": [
        "Music Festival India: (India)",
        "Sunburn Festival: (India)",
        "Lollapalooza India: (India)",
        "Coldplay India: (India)",
        "Diljit Dosanjh tour: (India)",
        "Karan Aujla concert: (India)"
    ],
    "Standup": [
        "Stand up comedy near me: (India)",
        "Comedy show tickets: (India)",
        "Comedy club: (India)",
        "Zakir Khan: (India)",
        "Samay Raina: (India)",
        "Abhishek Upmanyu: (India)",
        "Anubhav Singh Bassi: (India)"
    ],
    "Workshop": [
        "Pottery workshop: (India)",
        "Painting workshop: (India)"
    ]
}

# -----------------------------
# City payment power multiplier (heuristic)
# -----------------------------
city_multiplier = {
    "Mumbai": 1.30,
    "Delhi": 1.25,
    "Bengaluru": 1.20,
    "Bangalore": 1.20,
    "Gurgaon": 1.20,
    "Gurugram": 1.20,
    "Noida": 1.15,
    "Hyderabad": 1.15,
    "Pune": 1.15,
    "Chennai": 1.10,
    "Kolkata": 1.05,
    "Ahmedabad": 1.00,
    "Surat": 0.98,
    "Jaipur": 0.98,
    "Chandigarh": 1.05,
    "Lucknow": 0.95,
    "Indore": 0.95,
    "Vadodara": 0.95,
    "Bhopal": 0.92,
    "Nagpur": 0.92,
    "Coimbatore": 0.92,
    "Visakhapatnam": 0.92,
    "Patna": 0.85,
    "Ranchi": 0.85,
    "Raipur": 0.88,
    "Guwahati": 0.88,
    "Bhubaneswar": 0.90,
    "Kanpur": 0.88,
    "Varanasi": 0.88,
    "Ludhiana": 0.90,
    "Amritsar": 0.90,
    "Jalandhar": 0.90
}

# -----------------------------
# Base prices (category baseline)
# -----------------------------
base_price = {
    "Concert": 3500,
    "Standup": 1200,
    "Workshop": 800
}

# -----------------------------
# Inputs
# -----------------------------
col1, col2 = st.columns(2)
with col1:
    show_type = st.selectbox("Select Show Type", ["Concert", "Standup", "Workshop"])
with col2:
    city = st.selectbox(
        "Select City / Market",
        sorted(city_multiplier.keys())
    )

month_choice = st.selectbox(
    "Target Month",
    ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
)

price = st.slider("Your Planned Ticket Price (₹)", min_value=0, max_value=20000, value=2000, step=100)

month_map = {"Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,"Jul":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12}
target_month = month_map[month_choice]

# -----------------------------
# Demand calc (India-level)
# -----------------------------
keywords = category_map[show_type]
subset = df[(df["Keyword"].isin(keywords)) & (df["Month"] == target_month)]

avg_demand = float(subset["Demand_Index"].mean()) if len(subset) else 0.0
avg_trend = float(subset["Trend_Score"].mean()) if len(subset) else 0.0

mult = city_multiplier.get(city, 1.00)
city_adjusted_demand = avg_demand * mult

# -----------------------------
# NEW: Recommended Price formula
# -----------------------------
# demand_factor: neutral at 50, higher demand increases price
demand_factor = city_adjusted_demand / 50.0
demand_factor = max(0.70, min(demand_factor, 1.60))  # clamp

recommended_price = base_price[show_type] * mult * demand_factor
recommended_price = int(round(recommended_price / 100) * 100)  # round to nearest 100

# -----------------------------
# Recommendation (price vs recommended)
# -----------------------------
def price_action(demand, user_price, rec_price):
    # If demand is extremely low, pause regardless
    if demand < 25:
        return "REMOVE / PAUSE", "Demand is too low for this month. Pause or shift month/city."

    # Compare user price with recommended price
    diff = user_price - rec_price
    diff_pct = diff / max(rec_price, 1)

    if diff_pct < -0.15:
        return "INCREASE PRICE", f"Your price is low vs recommended ₹{rec_price}."
    elif diff_pct > 0.15:
        return "DECREASE PRICE", f"Your price is high vs recommended ₹{rec_price}."
    else:
        return "KEEP", f"Your price is close to recommended ₹{rec_price}."

action, reason = price_action(city_adjusted_demand, price, recommended_price)

# -----------------------------
# Display
# -----------------------------
st.subheader("📌 Demand Snapshot")
m1, m2, m3 = st.columns(3)
m1.metric("Avg Demand Index (Month)", round(avg_demand, 2))
m2.metric("City Multiplier", mult)
m3.metric("City Adjusted Demand", round(city_adjusted_demand, 2))

st.subheader("💰 Pricing Recommendation")
p1, p2 = st.columns(2)
p1.metric("Recommended Price (₹)", recommended_price)
p2.metric("Your Price (₹)", price)

st.subheader("✅ Final Recommendation")
st.write(f"**Action:** {action}")
st.write(f"**Reason:** {reason}")

# -----------------------------
# Plan table (optional)
# -----------------------------
st.divider()
st.subheader("📋 Plan Table")

if "plan" not in st.session_state:
    st.session_state.plan = pd.DataFrame(columns=[
        "Show_Type","City","Month","Your_Price","Recommended_Price",
        "Adj_Demand","Recommendation"
    ])

if st.button("➕ Add to Plan"):
    row = {
        "Show_Type": show_type,
        "City": city,
        "Month": month_choice,
        "Your_Price": price,
        "Recommended_Price": recommended_price,
        "Adj_Demand": round(city_adjusted_demand, 2),
        "Recommendation": action
    }
    st.session_state.plan = pd.concat([st.session_state.plan, pd.DataFrame([row])], ignore_index=True)
    st.success("Added.")

st.dataframe(st.session_state.plan, use_container_width=True)

st.download_button(
    "⬇️ Download Plan CSV",
    data=st.session_state.plan.to_csv(index=False).encode("utf-8"),
    file_name="show_plan.csv",
    mime="text/csv"
)


Overwriting app.py


In [51]:


!pip -q install streamlit
!pkill -f streamlit || true
!pkill -f cloudflared || true

import os, subprocess, time, re

# Quick checks
if not os.path.exists("app.py"):
    raise FileNotFoundError("app.py not found in /content. Run the app.py write cell first.")
if not os.path.exists("final_trends_seasonality.csv"):
    raise FileNotFoundError("final_trends_seasonality.csv not found in /content. Run Cell 1 first.")

# Start Streamlit
subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"],
    stdout=open("streamlit_logs.txt", "w"),
    stderr=subprocess.STDOUT
)

# Install cloudflared
subprocess.run(
    ["bash", "-lc", "wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared && chmod +x cloudflared"],
    check=True
)

# Start tunnel + print URL
proc = subprocess.Popen(
    ["./cloudflared", "tunnel", "--url", "http://localhost:8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

print("Starting tunnel... wait 5–15 seconds")
public_url = None

for _ in range(150):
    line = proc.stdout.readline().strip()
    if line:
        m = re.search(r"https://[a-zA-Z0-9\-]+\.trycloudflare\.com", line)
        if m:
            public_url = m.group(0)
            break
    time.sleep(0.2)

if public_url:
    print("\nOPEN THIS LINK:\n", public_url)
else:
    print("\nTunnel URL not found. Showing Streamlit logs:\n")
    !tail -n 80 streamlit_logs.txt


^C
^C
Starting tunnel... wait 5–15 seconds

OPEN THIS LINK:
 https://morgan-whose-eight-receives.trycloudflare.com
